In [1]:
%load_ext autoreload


In [2]:
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
''' Sn properties - Masses, Ratios, Cup Configurations, ... '''
from sn_config import *
'''Classes for Reading in the Data and applying Internal Normalisation'''
from nu_data_reduction import NU_data_read, int_norm, evaluation
from pylab import plot, show, savefig
from scipy import stats
import os.path
import re
#%pylab inline

In [4]:
path = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2015-03-25/"
path_sav = "/Volumes/friebelm/PhD/NU Plasma/Measurements/compilation/"
path_all = "/Volumes/friebelm/PhD/NU Plasma/Measurements/2016-08-30/"
files_start = 4547
files_end = 4713


In [5]:
# cup configuration
#cup_config = cycle_Sb
cup_config = cycles2
# Isotopes used for Interference correction
corr_isotopes_1 = {"Cd" : "111", "Te" : "126"}
corr_isotopes_2 = {"Cd" : "111","Te" : "125", "Xe" : "129"}
corr_isotopes_Sb = {"Te": "125"}
# Mass Range of cup configuration
mass_range = cycles2_mass_range
#isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124"], ["117", "118", "119", "122", "124"]]
isotopes = [["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]]
#isotopes = [["116", "117", "118", "119", "122", "124"]]
denom_isotope = "120"
isotope_column = ["111", "112", "114", "115", "116", "117", "118", "119", "122", "124", "125", "126", "129"]
data_sample_column = [(i + "/" + "120") for i in isotope_column]
print data_sample_column
#number of iterations for beta
iter_beta = 10

# Interference_corr on the denominator isotope
isotope_denom_corr = True

blk_corr = True

['111/120', '112/120', '114/120', '115/120', '116/120', '117/120', '118/120', '119/120', '122/120', '124/120', '125/120', '126/120', '129/120']


In [6]:
def mad_based_outlier(points, thresh=3.5):
        """
        Returns a boolean array with True if points are outliers and False
        otherwise.

        Parameters:
        -----------
            points : An numobservations by numdimensions array of observations
            thresh : The modified z-score to use as a threshold. Observations with
                a modified z-score (based on the median absolute deviation) greater
                than this value will be classified as outliers.

        Returns:
        --------
            mask : A numobservations-length boolean array.

        References:
        ----------
            Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
            Handle Outliers", The ASQC Basic References in Quality Control:
            Statistical Techniques, Edward F. Mykytka, Ph.D., Editor.
        """
        if len(points.shape) == 1:
            points = points[:,None]
        median = np.median(points, axis=0)
        diff = np.sum((points - median)**2, axis=-1)
        diff = np.sqrt(diff)
        med_abs_deviation = np.median(diff)

        modified_z_score = 0.6745 * diff / med_abs_deviation

        return modified_z_score > thresh

#outlier rejection
def mad_outlier_rejection(df):
    data_sample_outlier = pd.DataFrame()
    columns = df.columns.tolist()
    for column in columns:
        data_sample_outlier[column] = df[column].where(~mad_based_outlier(df[column]), other=np.NaN)
    return data_sample_outlier

In [7]:
filenames = os.listdir(path_all)

In [8]:
blk_ls = []
sample_ls = []
std_ls = []
for i in filenames:
    sample = re.search(r""+"Data_"+"(.*).csv", i)
    if sample == None:
        None
    else:
        sample = sample.group(1)
        df = NU_data_read(path_all, sample, cup_config)
        sample_name = df.extract_metadata(sample, "Sample Name")
        if sample_name == "blank sol" or sample_name == "wash" or sample_name == "wash clean" or sample_name == "Teflon blk":
            blk_ls.append(sample)
        elif sample_name == "SPEX 100ppb" or sample_name == "NIST 100ppb":
            std_ls.append(sample)
        else:
            sample_ls.append(sample)

In [13]:
columns = ["cycle", "sample", "date","H10 (1)", "H9 (1)", "H8 (1)", "H7 (1)", "H6 (1)", "H5 (1)", "H4 (1)", "H3 (1)", "H2 (1)", "H1 (1)", "Ax (1)", "L1 (1)", "L2 (1)", "L3 (1)", "L4 (1)", 
             "H10 (2)", "H9 (2)", "H8 (2)", "H7 (2)", "H6 (2)", "H5 (2)", "H4 (2)", "H3 (2)", "H2 (2)", "H1 (2)", "Ax (2)", "L1 (2)", "L2 (2)", "L3 (2)", "L4 (2)"]
df_all = pd.DataFrame(columns = columns)
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Data_" + str(sample) + ".csv"
    data = pd.read_csv(path_all + datafile, skiprows=58, index_col='Cycle')
    if "H8 (2)" in data.columns:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_2 = data.ix[:,"H10 (2)":"L4 (2)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_zero.columns = data_2.columns
        data_2_corr = data_2 - data_zero
        data_2_corr["cycle"] = data_2_corr.index.values
        data_1_corr = data_1_corr.merge(data_2_corr)
        data_1_corr = mad_outlier_rejection(data_1_corr)
    else:
        data_zero = data.ix[:,"H10 (Z1)":"L4 (Z1)"]
        data_1 = data.ix[:,"H10 (1)":"L4 (1)"]
        data_zero.columns = data_1.columns
        data_1_corr = data_1 - data_zero
        data_1_corr["cycle"] = data_1_corr.index.values
        data_1_corr = mad_outlier_rejection(data_1_corr)
        
    sample_name = df.extract_metadata(sample, "Sample Name")
    date = df.extract_metadata(sample, "Date")
    starttime = df.extract_metadata(sample, "Start Time")
    data_1_corr["date"] = datetime.strptime(date+starttime, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    df_all = pd.concat([df_all,data_1_corr], axis=0, ignore_index=True)
    #df_all = df_all.append(data_1_corr)
    df_all = df_all[columns]
df_all.set_index("date", inplace = True)
df_all

,cycle,sample,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),...,H5 (2),H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2)
date,,,,,,,,,,,,,,,,,,,,,
2016-08-30 17:17:00,1,blank sol,0,0,0.001807,0.001393,0.010537,0.002494,0.006916,0.002183,...,0.000091,6.461000e-07,0.001870,2.112380e-05,0.001413,0.000039,0.010954,0.002574,0.007172,0.002257
2016-08-30 17:17:00,2,blank sol,0,0,0.002019,0.001549,0.011714,0.002783,0.007770,0.002435,...,0.000072,1.777820e-05,0.002093,5.127520e-05,0.001587,0.000054,0.011989,0.002863,0.007948,0.002512
2016-08-30 17:17:00,3,blank sol,0,0,0.001912,0.001466,0.011140,0.002625,0.007340,0.002331,...,0.000080,2.563990e-05,0.001861,2.901850e-05,0.001403,0.000059,0.010768,0.002525,0.007089,0.002242
2016-08-30 17:17:00,4,blank sol,0,0,0.002245,0.001720,0.012838,0.003057,0.008595,0.002698,...,0.000091,2.562470e-05,NaN,5.701260e-05,0.001704,0.000051,0.013102,0.003094,0.008695,0.002752
2016-08-30 17:17:00,5,blank sol,0,0,0.001873,0.001427,0.010837,0.002572,0.007148,0.002248,...,0.000080,-2.213600e-06,0.001739,4.051020e-05,0.001336,0.000034,0.010159,0.002376,0.006610,0.002076
2016-08-30 17:17:00,6,blank sol,0,0,0.001843,0.001409,0.010795,0.002540,0.007129,0.002237,...,0.000110,5.206990e-05,0.001789,4.194620e-05,0.001314,0.000052,0.010325,0.002397,0.006704,0.002130
2016-08-30 17:17:00,7,blank sol,0,0,0.001636,0.001265,0.009795,0.002340,0.006435,0.002010,...,0.000077,2.850800e-05,0.001633,3.979970e-05,0.001252,0.000048,0.009674,0.002227,0.006243,0.001962
2016-08-30 17:17:00,8,blank sol,0,0,0.001884,0.001427,0.010970,0.002610,0.007251,0.002306,...,0.000083,2.849820e-05,0.001811,4.553590e-05,0.001356,0.000050,0.010436,0.002414,0.006723,0.002101
2016-08-30 17:17:00,9,blank sol,0,0,0.001740,0.001350,0.010188,0.002412,0.006711,0.002109,...,0.000057,1.421960e-05,0.001882,NaN,0.001432,0.000069,0.011016,0.002580,0.007177,0.002255


In [14]:
def mean(x):
    mean = np.nanmean(x) 
    return mean

df_all_mean = df_all.drop("cycle", axis=1).groupby(df_all.index).agg(mean)
df_all_mean["sample"] = df_all[df_all["cycle"] == 1]["sample"]
df_all_mean
#(df_all_mean["H6 (1)"].mean() - (df_all_mean["H4 (1)"].mean()/24.22*32.6))*1000

,H10 (1),H9 (1),H8 (1),H7 (1),H6 (1),H5 (1),H4 (1),H3 (1),H2 (1),H1 (1),...,H4 (2),H3 (2),H2 (2),H1 (2),Ax (2),L1 (2),L2 (2),L3 (2),L4 (2),sample
date,,,,,,,,,,,,,,,,,,,,,
2016-08-30 16:25:00,0,0,0.002268,0.001717,0.013021,0.003135,0.008678,0.002733,0.005266,0.000115,...,0.000023,0.002314,0.000038,0.001752,0.000046,0.013228,0.003179,0.008811,0.002829,blank sol
2016-08-30 16:51:00,0,0,0.001888,0.001470,0.011200,0.002590,0.007375,0.002319,0.004514,0.000128,...,0.000026,0.001928,0.000045,0.001488,0.000047,0.011242,0.002666,0.007396,0.002327,blank sol
2016-08-30 17:17:00,0,0,0.001867,0.001420,0.010827,0.002539,0.007155,0.002256,0.004334,0.000100,...,0.000022,0.001791,0.000037,0.001390,0.000048,0.010707,0.002508,0.006999,0.002194,blank sol
2016-08-30 18:42:00,0,0,0.001805,0.001379,0.010643,0.002481,0.006958,0.002195,0.004258,0.000119,...,0.000016,0.001817,0.000034,0.001408,0.000042,0.010790,0.002512,0.007055,0.002214,blank sol
2016-08-30 20:07:00,0,0,0.001711,0.001307,0.010112,0.002350,0.006596,0.002080,0.004040,0.000109,...,0.000024,0.001823,0.000026,0.001337,0.000048,0.010269,0.002512,0.007016,0.002103,blank sol
2016-08-30 21:31:00,0,0,0.001444,0.001093,0.008714,0.001999,0.005594,0.001767,0.003461,0.000080,...,0.000014,0.001483,0.000024,0.001120,0.000062,0.008919,0.002044,0.005736,0.001815,blank sol
2016-08-30 22:55:00,0,0,0.001466,0.001105,0.008566,0.002000,0.005599,0.001774,0.003468,0.000093,...,0.000016,0.001360,0.000030,0.001021,0.000042,0.008187,0.001810,0.005181,0.001584,blank sol
2016-08-31 00:19:00,0,0,0.001266,0.000940,0.007699,0.001700,0.004863,0.001544,0.002975,0.000081,...,0.000030,0.001270,0.000035,0.000962,0.000048,0.007730,0.001741,0.004862,0.001537,blank sol
2016-08-31 01:42:00,0,0,0.001254,0.000954,0.007644,0.001731,0.004827,0.001532,0.003007,0.000098,...,0.000016,0.001288,0.000028,0.000994,0.000040,0.007800,0.001792,0.004978,0.001569,blank sol


In [40]:
df_peak = pd.DataFrame(columns = ["date", "sample", "peak_center_mass"])
data_1_corr = pd.DataFrame()
for sample in blk_ls:
    df = NU_data_read(path_all, sample, cup_config)
    datafile = "Results_" + str(sample) + ".txt"
    sample_name = df.extract_metadata(sample, "Sample Name:")
    date = df.extract_metadata(sample, "Date and Time:")
    peak_center = df.extract_metadata(sample, "Axial Mass(es) used:")
    data_1_corr["date"] = datetime.strptime(date, '%d/%m/%Y%H:%M')
    data_1_corr["sample"] = sample_name
    data_1_corr["peak_center_mass"] = peak_center
    df_peak = pd.concat([df_peak,data_1_corr], axis=0, ignore_index=True)

df_peak

UnboundLocalError: local variable 'metadata' referenced before assignment

In [15]:
df_all.to_csv(path_all + "blanks_outlier_corr.csv", mode='w', header=True)

In [16]:
df_all_mean.to_csv(path_all + "blanks_mean_outlier_corr.csv", mode='w', header=True)

In [229]:
df_internal_norm_mean_sd.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_mean_no_bgd_corr.csv")

In [389]:
epsilon.to_csv(path + "Sn_H8_L4_2_cycles_internal_norm_epsilon_bgd_corr.csv")